## Hydra-zen

### Generate Hydra compatible cfg

In [1]:
import omegaconf
import pyrootutils

root = pyrootutils.setup_root("/root/workspace/EP2", pythonpath=True)
cfg = omegaconf.OmegaConf.load(root / "configs" / "model" / "mnist_eqprop.yaml")

In [3]:
omegaconf.OmegaConf.register_new_resolver("double", lambda condition, false_value: 2 * false_value if condition else false_value)

In [4]:
cfg.net.input_size

1568

In [13]:
print(omegaconf.OmegaConf.to_yaml(cfg))


_target_: src.models.eqprop_module.EqPropLitModule
optimizer:
  _target_: torch.optim.Adam
  _partial_: true
  lr: 0.001
  weight_decay: 0.0
scheduler:
  _target_: torch.optim.lr_scheduler.ReduceLROnPlateau
  _partial_: true
  mode: min
  factor: 0.1
  patience: 10
net:
  _target_: src.models.components.eqprop_backbone.AnalogEP2
  _partial_: true
  input_size: ${double:${double_input}}
  lin1_size: 128
  output_size: 10
  double_input: true
  double_output: true
  positive_w: true
  bias: false
  clip_weights: true



In [13]:
import hydra

hydra.instan(cfg)

/tmp/ipykernel_860/528185045.py:3: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(cfg)


TypeError: expected str, bytes or os.PathLike object, not DictConfig

In [7]:
import hydra_zen

hydra_zen.instantiate(cfg)

InstantiationException: Error locating target 'src.models.eqprop_module.EqPropLitModule', set env var HYDRA_FULL_ERROR=1 to see chained exception.

### Make Hierachial configs

see https://github.com/mit-ll-responsible-ai/hydra-zen-examples

In [ ]:
import math

import pytorch_lightning as pl
import torch as tr
from hydra_zen import builds, instantiate, make_config, make_custom_builds_fn
from torch.optim import Adam
from torch.utils.data import DataLoader
from zen_model import UniversalFuncModule, single_layer_nn

pbuilds = make_custom_builds_fn(zen_partial=True, populate_full_signature=True)

OptimConf = pbuilds(Adam)

LoaderConf = pbuilds(DataLoader, batch_size=25, shuffle=True, drop_last=True)

ModelConf = builds(single_layer_nn, num_neurons=10)

# configure our lightning module
LitConf = pbuilds(
    UniversalFuncModule,
    model=ModelConf,
    target_fn=tr.cos,
    training_domain=builds(tr.linspace, start=-2 * math.pi, end=2 * math.pi, steps=1000),
)

TrainerConf = builds(pl.Trainer, max_epochs=100)

ExperimentConfig = make_config(
    optim=OptimConf,
    dataloader=LoaderConf,
    lit_module=LitConf,
    trainer=TrainerConf,
    seed=1,
)


def task_function(cfg):
    # cfg: ExperimentConfig
    pl.seed_everything(cfg.seed)

    obj = instantiate(cfg)

    # finish instantiating the lightning module, data-loader, and optimizer
    lit_module = obj.lit_module(dataloader=obj.dataloader, optim=obj.optim)

    # train the model
    obj.trainer.fit(lit_module)

    # evaluate the model over the domain to assess the fit
    data = lit_module.training_domain
    final_eval = lit_module.forward(data.reshape(-1, 1))
    final_eval = final_eval.detach().cpu().numpy().ravel()

    # return the final evaluation of our model:
    # a shape-(N,) numpy-array
    return final_eval

# Hydra

In [ ]:
import hydra

@hydra.main(config_path="configs", config_name="model/mnist")

In [ ]:
import unittest
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from src.models.eqprop_module import EqPropLitModule
from src.models.components.eqprop_backbone import AnalogEP2

class TestMnistEqprop(unittest.TestCase):
    def setUp(self):
        self.optimizer = optim.Adam(lr=0.001, weight_decay=0.0)
        self.scheduler = lr_scheduler.ReduceLROnPlateau(mode='min', factor=0.1, patience=10)
        self.net = AnalogEP2(input_size=2, lin1_size=128, output_size=10)
        self.model = EqPropLitModule(self.net, self.optimizer, self.scheduler)

    def test_optimizer(self):
        self.assertIsInstance(self.model.optimizer, optim.Adam)
        self.assertEqual(self.model.optimizer.defaults['lr'], 0.001)
        self.assertEqual(self.model.optimizer.defaults['weight_decay'], 0.0)

    def test_scheduler(self):
        self.assertIsInstance(self.model.scheduler, lr_scheduler.ReduceLROnPlateau)
        self.assertEqual(self.model.scheduler.mode, 'min')
        self.assertEqual(self.model.scheduler.factor, 0.1)
        self.assertEqual(self.model.scheduler.patience, 10)

    def test_net(self):
        self.assertIsInstance(self.model.net, AnalogEP2)
        self.assertEqual(self.model.net.input_size, 2)
        self.assertEqual(self.model.net.lin1_size, 128)
        self.assertEqual(self.model.net.output_size, 10)

    def test_double_input(self):
        self.assertEqual(self.model.net.input_size, 4)

    def test_double_output(self):
        self.assertEqual(self.model.net.output_size, 20)

    def test_positive_w(self):
        self.assertTrue((self.model.net.w > 0).all())

    def test_bias(self):
        self.assertIsNone(self.model.net.b)

    def test_clip_weights(self):
        self.assertTrue((self.model.net.w.abs() <= 1).all())

if __name__ == '__main__':
    unittest.main()

In [ ]:
import pytest
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from src.models.eqprop_module import EqPropLitModule
from src.models.components.eqprop_backbone import AnalogEP2

@pytest.fixture
def eqprop_model():
    optimizer = optim.Adam(lr=0.001, weight_decay=0.0)
    scheduler = lr_scheduler.ReduceLROnPlateau(mode='min', factor=0.1, patience=10)
    net = AnalogEP2(input_size=2, lin1_size=128, output_size=10)
    model = EqPropLitModule(net, optimizer, scheduler)
    return model

def test_optimizer(eqprop_model):
    assert isinstance(eqprop_model.optimizer, optim.Adam)
    assert eqprop_model.optimizer.defaults['lr'] == 0.001
    assert eqprop_model.optimizer.defaults['weight_decay'] == 0.0

def test_scheduler(eqprop_model):
    assert isinstance(eqprop_model.scheduler, lr_scheduler.ReduceLROnPlateau)
    assert eqprop_model.scheduler.mode == 'min'
    assert eqprop_model.scheduler.factor == 0.1
    assert eqprop_model.scheduler.patience == 10

def test_net(eqprop_model):
    assert isinstance(eqprop_model.net, AnalogEP2)
    assert eqprop_model.net.input_size == 2
    assert eqprop_model.net.lin1_size == 128
    assert eqprop_model.net.output_size == 10

def test_double_input(eqprop_model):
    assert eqprop_model.net.input_size == 4

def test_double_output(eqprop_model):
    assert eqprop_model.net.output_size == 20

def test_positive_w(eqprop_model):
    assert (eqprop_model.net.w > 0).all()

def test_bias(eqprop_model):
    assert eqprop_model.net.b is None

def test_clip_weights(eqprop_model):
    assert (eqprop_model.net.w.abs() <= 1).all()

In [ ]:
import hydra
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from src.models.eqprop_module import EqPropLitModule

@hydra.main(config_path='conf', config_name='config')
def test_eqprop(cfg):
    optimizer = optim.Adam(lr=cfg.optimizer.lr, weight_decay=cfg.optimizer.weight_decay)
    scheduler = lr_scheduler.ReduceLROnPlateau(mode=cfg.scheduler.mode, factor=cfg.scheduler.factor, patience=cfg.scheduler.patience)
    net = hydra.utils.instantiate(cfg.net)
    model = EqPropLitModule(net, optimizer, scheduler)

    assert isinstance(model.optimizer, optim.Adam)
    assert model.optimizer.defaults['lr'] == cfg.optimizer.lr
    assert model.optimizer.defaults['weight_decay'] == cfg.optimizer.weight_decay

    assert isinstance(model.scheduler, lr_scheduler.ReduceLROnPlateau)
    assert model.scheduler.mode == cfg.scheduler.mode
    assert model.scheduler.factor == cfg.scheduler.factor
    assert model.scheduler.patience == cfg.scheduler.patience

    assert isinstance(model.net, hydra.types.Instantiable)
    assert model.net.input_size == cfg.net.input_size
    assert model.net.lin1_size == cfg.net.lin1_size
    assert model.net.output_size == cfg.net.output_size

    if cfg.double_input:
        assert model.net.input_size == cfg.net.input_size * 2
    else:
        assert model.net.input_size == cfg.net.input_size

    if cfg.double_output:
        assert model.net.output_size == cfg.net.output_size * 2
    else:
        assert model.net.output_size == cfg.net.output_size

    if cfg.positive_w:
        assert (model.net.w > 0).all()
    else:
        assert (model.net.w <= 0).any()

    if cfg.bias:
        assert model.net.b is not None
    else:
        assert model.net.b is None

    if cfg.clip_weights:
        assert (model.net.w.abs() <= 1).all()
    else:
        assert (model.net.w.abs() > 1).any()

if __name__ == '__main__':
    test_eqprop()